# 🧬 Discovering Emerging Topics in Drug Discovery Research using BERTopic

## 1️⃣Import dependencies

In [21]:
import os
import re, spacy
import pandas as pd
import os, random
import numpy as np
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)

from Bio import Entrez, Medline
from bertopic import BERTopic
from transformers import pipeline
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer


## 2️⃣ Data Collection
I used PubMed  to fetch abstracts on topics like 'drug discovery', 'AI drug design', and 'molecular docking'.
Store as `data/raw_publications.csv`. 

In [ ]:
Entrez.email = "paulafredrick26@gmail.com"  # already set
# 1. Search PubMed
handle = Entrez.esearch(
    db="pubmed",
    term=query,
    retmax=max_results,
    sort="relevance"
)
search_results = Entrez.read(handle)
handle.close()

pmids = search_results["IdList"]
print("Number of PMIDs found:", len(pmids))

# 2. Fetch details
handle = Entrez.efetch(
    db="pubmed",
    id=",".join(pmids),
    rettype="medline",
    retmode="text"
)
records = list(Medline.parse(handle))
handle.close()

# 3. Build DataFrame
rows = []
for rec in records:
    abstract = rec.get("AB", "")
    if not abstract:
        continue
    rows.append({
        "pmid": rec.get("PMID", ""),
        "title": rec.get("TI", ""),
        "abstract": abstract,
        "journal": rec.get("JT", ""),
        "year": rec.get("DP", "")[:4]
    })

df = pd.DataFrame(rows)
print(df.shape)
df.head()
    
# 4. Save CSV
os.makedirs("data", exist_ok=True)
df.to_csv("data/raw_publications.csv", index=False)
print("Saved to data/raw_publications.csv")

Number of PMIDs found: 1000
(976, 5)
Saved to data/raw_publications.csv


### Start from Here!!

## 3️⃣ Data Cleaning & Preprocessing

In [22]:
nlp = spacy.load("en_core_web_sm")

def clean_text(text):
    text = str(text)
    # collapse whitespace
    text = re.sub(r"\s+", " ", text)
    # remove punctuation, make lowercase
    text = re.sub(r"[^\w\s]", "", text.lower())
    doc = nlp(text)
    tokens = [
        token.lemma_
        for token in doc
        if not token.is_stop and token.is_alpha
    ]
    return " ".join(tokens)

df = pd.read_csv("data/raw_publications.csv")
df["clean_text"] = df["abstract"].apply(clean_text)
df.head()

,pmid,title,abstract,journal,year,clean_text
0,31487867,Molecular Docking: Shifting Paradigms in Drug ...,Molecular docking is an established in silico ...,International journal of molecular sciences,2019,molecular docking establish silico structureba...
1,26205061,Molecular docking and structure-based drug des...,Pharmaceutical research has successfully incor...,"Molecules (Basel, Switzerland)",2015,pharmaceutical research successfully incorpora...
2,38594926,The Art and Science of Molecular Docking.,Molecular docking has become an essential part...,Annual review of biochemistry,2024,molecular docking essential structural biologi...
3,34560276,Machine-learning methods for ligand-protein mo...,Artificial intelligence (AI) is often presente...,Drug discovery today,2022,artificial intelligence ai present new industr...
4,34147204,Use of molecular docking computational tools i...,Molecular docking has become an important comp...,Progress in medicinal chemistry,2021,molecular docking important component drug dis...


## 4️⃣ Embedding Generation

In [23]:
import torch
torch.manual_seed(SEED)
model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(
    df['clean_text'], 
    show_progress_bar=True, 
    convert_to_numpy=True
)

Batches:   0%|          | 0/31 [00:00<?, ?it/s]

## 5️⃣ Hyperparameter Tuning

In [24]:
# Experiment
umap_params = [
    {"n_neighbors": 5,  "n_components": 5},
    {"n_neighbors": 10, "n_components": 5},
    {"n_neighbors": 15, "n_components": 5},
    {"n_neighbors": 5,  "n_components": 10},
    {"n_neighbors": 10, "n_components": 10},
    {"n_neighbors": 15, "n_components": 10},
]

hdbscan_params = [
    {"min_cluster_size": 8},
    {"min_cluster_size": 12},
]

In [25]:
results = []

for u in umap_params:
    for h in hdbscan_params:

        umap_model = UMAP(
            n_neighbors=u["n_neighbors"],
            n_components=u["n_components"],
            min_dist=0.0,
            metric="cosine",
            random_state=42
        )

        hdbscan_model = HDBSCAN(
            min_cluster_size=h["min_cluster_size"],
            metric="euclidean",
            cluster_selection_method="eom",
            prediction_data=True
        )

        topic_model = BERTopic(
            umap_model=umap_model,
            hdbscan_model=hdbscan_model,
            verbose=False
        )

        topics, _ = topic_model.fit_transform(df["clean_text"], embeddings)

        info = topic_model.get_topic_info()

        n_topics = len(info[info.Topic != -1])
        noise_docs = info[info.Topic == -1]["Count"].values[0] if -1 in info.Topic.values else 0

        results.append({
            "n_neighbors": u["n_neighbors"],
            "n_components": u["n_components"],
            "min_cluster_size": h["min_cluster_size"],
            "n_topics": n_topics,
            "noise_docs": noise_docs
        })


In [15]:
results_df = pd.DataFrame(results)
results_df.sort_values("n_topics", ascending=False)

,n_neighbors,n_components,min_cluster_size,n_topics,noise_docs
8,10,10,8,23,287
2,10,5,8,22,345
4,15,5,8,21,362
0,5,5,8,21,222
6,5,10,8,19,238
10,15,10,8,17,279
11,15,10,12,10,249
3,10,5,12,10,236
5,15,5,12,9,263
7,5,10,12,6,64


## 6️⃣ Topic Modelling with BERTopic

In [26]:
umap_model = UMAP(n_neighbors=10, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=12, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

topic_model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model)
topics, probs = topic_model.fit_transform(df['clean_text'], embeddings)
topic_model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,236,-1_compound_inhibitor_molecular_study,"[compound, inhibitor, molecular, study, drug, ...",[background delta play key role bcell signal t...
1,0,366,0_drug_docking_discovery_method,"[drug, docking, discovery, method, molecular, ...",[protein threedimensional structure prediction...
2,1,106,1_drug_virus_protease_antiviral,"[drug, virus, protease, antiviral, viral, comp...",[emergence new variant raise concern effective...
3,2,58,2_compound_drug_tuberculosis_inhibitor,"[compound, drug, tuberculosis, inhibitor, stud...",[tuberculosis tb infectious disease cause numb...
4,3,48,3_ad_disease_compound_inhibitor,"[ad, disease, compound, inhibitor, alzheimer, ...",[discovery novel multifunctional inhibitor tar...
5,4,40,4_kinase_inhibitor_compound_cell,"[kinase, inhibitor, compound, cell, cancer, po...",[kinase key target approximately breast cancer...
6,5,30,5_inhibitor_bind_cancer_molecular,"[inhibitor, bind, cancer, molecular, study, co...",[failure chemotherapy treatment carcinoma main...
7,6,30,6_alphaglucosidase_compound_study_alphaamylase,"[alphaglucosidase, compound, study, alphaamyla...",[diabete mellitus dm multifactorial lifethreat...
8,7,24,7_gpcrs_gpcr_ligand_receptor,"[gpcrs, gpcr, ligand, receptor, agonist, allos...",[gproteincouple receptor gpcrs tractable drug ...
9,8,23,8_antimalarial_malaria_drug_parasite,"[antimalarial, malaria, drug, parasite, new, a...",[identify novel lead compound drug discovery c...


##### Topic evolution over time

In [27]:
topics_over_time = topic_model.topics_over_time(
    df["clean_text"],
    df["year"]
)
topics_over_time 

,Topic,Words,Frequency,Timestamp
0,-1,"inhibitor, study, compound, covalent, design",14,2015
1,0,"docking, drug, discovery, screening, molecular",32,2015
2,1,"virus, dengue, hbv, pr, protease",3,2015
3,2,"tuberculosis, inhibitor, derivative, novel, oh...",4,2015
4,3,"buche, azaphenothiazine, ache, inhibitor, synt...",2,2015
...,...,...,...,...
106,2,"compound, tuberculosis, drug, tb, study",6,2025
107,3,"fisher, target, neurodegenerative, candidate, ...",2,2025
108,4,"aak, kinase, stk, inhibitor, compound",4,2025
109,5,"caffeic, phytoconstituent, ehretia, acid, energy",1,2025


## 7️⃣ Zero-Shot Topic Labeling for Enhanced Interpretability
To improve interpretability of the unsupervised BERTopic results, I applied a zero-shot classification model (BART-MNLI) to automatically assign semantic labels to the representative documents of each topic.

In [30]:
classifier = pipeline("zero-shot-classification",
                      model="valhalla/distilbart-mnli-12-1")  

candidate_labels = [
    "molecular docking and virtual screening",
    "antiviral drug discovery (protease inhibitors)",
    "cancer therapeutics (kinase inhibitors, anticancer compounds)",
    "neurodegenerative disease drug discovery (Alzheimer’s)",
    "antibiotic discovery and resistance (bacterial targets)",
    "tuberculosis drug discovery",
    "antimalarial drug discovery",
    "GPCR drug discovery (ligands and receptors)",
    "metabolic disease therapeutics (diabetes enzymes)",
    "protein-protein interaction inhibitors",
    "drug target identification and validation"
]

config.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/890M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/890M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


In [31]:
topics = []
texts = []

topic_representatives = topic_model.get_representative_docs()
for topic, docs in topic_representatives.items():
    if topic == -1:
        continue
    topics.append(topic)
    texts.append(docs[0][:800])

outputs = classifier(texts, candidate_labels, batch_size=8)

zero_shot_labels = {t: out["labels"][0] for t, out in zip(topics, outputs)}
zero_shot_labels

{0: 'drug target identification and validation',
 1: 'molecular docking and virtual screening',
 2: 'tuberculosis drug discovery',
 3: 'drug target identification and validation',
 4: 'molecular docking and virtual screening',
 5: 'molecular docking and virtual screening',
 6: 'molecular docking and virtual screening',
 7: 'drug target identification and validation',
 8: 'antimalarial drug discovery',
 9: 'drug target identification and validation'}

In [33]:
info = topic_model.get_topic_info().copy()
info["zero_shot_label"] = info["Topic"].map(zero_shot_labels)
info

,Topic,Count,Name,Representation,Representative_Docs,zero_shot_label
0,-1,236,-1_compound_inhibitor_molecular_study,"[compound, inhibitor, molecular, study, drug, ...",[background delta play key role bcell signal t...,NaN
1,0,366,0_drug_docking_discovery_method,"[drug, docking, discovery, method, molecular, ...",[protein threedimensional structure prediction...,drug target identification and validation
2,1,106,1_drug_virus_protease_antiviral,"[drug, virus, protease, antiviral, viral, comp...",[emergence new variant raise concern effective...,molecular docking and virtual screening
3,2,58,2_compound_drug_tuberculosis_inhibitor,"[compound, drug, tuberculosis, inhibitor, stud...",[tuberculosis tb infectious disease cause numb...,tuberculosis drug discovery
4,3,48,3_ad_disease_compound_inhibitor,"[ad, disease, compound, inhibitor, alzheimer, ...",[discovery novel multifunctional inhibitor tar...,drug target identification and validation
5,4,40,4_kinase_inhibitor_compound_cell,"[kinase, inhibitor, compound, cell, cancer, po...",[kinase key target approximately breast cancer...,molecular docking and virtual screening
6,5,30,5_inhibitor_bind_cancer_molecular,"[inhibitor, bind, cancer, molecular, study, co...",[failure chemotherapy treatment carcinoma main...,molecular docking and virtual screening
7,6,30,6_alphaglucosidase_compound_study_alphaamylase,"[alphaglucosidase, compound, study, alphaamyla...",[diabete mellitus dm multifactorial lifethreat...,molecular docking and virtual screening
8,7,24,7_gpcrs_gpcr_ligand_receptor,"[gpcrs, gpcr, ligand, receptor, agonist, allos...",[gproteincouple receptor gpcrs tractable drug ...,drug target identification and validation
9,8,23,8_antimalarial_malaria_drug_parasite,"[antimalarial, malaria, drug, parasite, new, a...",[identify novel lead compound drug discovery c...,antimalarial drug discovery


In [34]:
import json
with open("data/zero_shot_labels.json", "w") as f:
    json.dump(zero_shot_labels, f, indent=2)

info.to_csv("data/topic_info_with_zero_shot.csv", index=False)

## 8 Evaluation & Visualization

In [40]:
topic_model.visualize_topics()
topic_model.visualize_barchart()
topic_model.visualize_hierarchy()

df["year"] = pd.to_numeric(df["year"], errors="coerce")

# Optional: drop rows without year
df_time = df.dropna(subset=["year"]).copy()
df_time["year"] = df_time["year"].astype(int)

# Compute topic evolution
topics_over_time = topic_model.topics_over_time(
    df_time["clean_text"],
    df_time["year"]
)

# Visualize
topic_model.visualize_topics_over_time(topics_over_time)


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hoverinfo': 'text',
              'hovertext': [<b>Topic 0</b><br>Words: docking, drug, discovery,
                            screening, molecular, <b>Topic 0</b><br>Words: docking,
                            drug, ligand, discovery, prediction, <b>Topic
                            0</b><br>Words: drug, discovery, docking, target,
                            screening, <b>Topic 0</b><br>Words: drug, docking,
                            ligand, method, discovery, <b>Topic 0</b><br>Words:
                            drug, docking, discovery, molecular, method, <b>Topic
                            0</b><br>Words: drug, method, docking, discovery,
                            molecular, <b>Topic 0</b><br>Words: drug, docking,
                            molecular, discovery, target, <b>Topic 0</b><br>Words:
                            drug, docking, discovery, method, ligand, <b>Topic
                            0</b><br>Words: drug, docking, discovery, method,
                            ligand, <b>Topic 0</b><br>Words: drug, docking,
                            discovery, molecular, model, <b>Topic 0</b><br>Words:
                            drug, method, computational, docking, design],
              'marker': {'color': '#E69F00'},
              'mode': 'lines',
              'name': '0_drug_docking_discovery_method',
              'type': 'scatter',
              'x': {'bdata': '3wfgB+EH4gfjB+QH5QfmB+cH6AfpBw==', 'dtype': 'i2'},
              'y': {'bdata': 'IBUVKhwjJR8lRws=', 'dtype': 'i1'}},
             {'hoverinfo': 'text',
              'hovertext': [<b>Topic 1</b><br>Words: virus, dengue, hbv, pr,
                            protease, <b>Topic 1</b><br>Words: zika, rna, virus,
                            tar, epitope, <b>Topic 1</b><br>Words: hcv, dengue,
                            drug, phytochemical, study, <b>Topic 1</b><br>Words:
                            influenza, virus, antiviral, ipnv, polymerase, <b>Topic
                            1</b><br>Words: antiviral, virus, drug, flavonoid,
                            activity, <b>Topic 1</b><br>Words: drug, virus,
                            antiviral, protease, zikv, <b>Topic 1</b><br>Words:
                            protease, drug, virus, compound, viral, <b>Topic
                            1</b><br>Words: drug, viral, virus, inhibitor,
                            compound, <b>Topic 1</b><br>Words: protease, mpro,
                            main, drug, eugenol, <b>Topic 1</b><br>Words: drug,
                            compound, virus, viral, protease, <b>Topic
                            1</b><br>Words: rdrp, virus, antiviral, drug, compound],
              'marker': {'color': '#56B4E9'},
              'mode': 'lines',
              'name': '1_drug_virus_protease_antiviral',
              'type': 'scatter',
              'x': {'bdata': '3wfgB+EH4gfjB+QH5QfmB+cH6AfpBw==', 'dtype': 'i2'},
              'y': {'bdata': 'AwUFBQMUGw4GDgQ=', 'dtype': 'i1'}},
             {'hoverinfo': 'text',
              'hovertext': [<b>Topic 2</b><br>Words: tuberculosis, inhibitor,
                            derivative, novel, ohydroxybenzylamine, <b>Topic
                            2</b><br>Words: glycosyltransferase, bacterial,
                            compound, disaccharide, ortholog, <b>Topic
                            2</b><br>Words: dna, mesotrypsin, inhibitor, compound,
                            drug, <b>Topic 2</b><br>Words: antibacterial, gyrase,
                            antimicrobial, dna, fad, <b>Topic 2</b><br>Words: kb,
                            compound, phosphotransferase, inhibitor, epepn,
                            <b>Topic 2</b><br>Words: tb, drug, antibiotic, study,
                            shikimate, <b>Topic 2</b><br>Words: study, drug,
                            compound, kinetoplastid, biofilm, <b>Topic
                            2</b><br>Words: drugability, mtb, antitb, profile,
                            glmu,

## 8️⃣ Results & Insights
- List and interpret top topics.
- Identify emerging or declining research themes.


## 9️⃣ Conclusion
- Summarize key findings.
- Highlight business and scientific relevance.
- Suggest future work (e.g., comparing journals, regions, or institutions).
